In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [3]:
# selenium version 확인
import selenium
print(f'selenium version: {selenium.__version__}')

selenium version: 4.14.0


In [5]:
site = "https://www.instagram.com/"
driver = webdriver.Chrome()
driver.get(site)

# 로그인 버튼 찾기 (XPath 사용) | ts-아래처럼 기다리는 처리하지 않아 오류 발생했음
login_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//button[.//div[text()="로그인"]]')))

# username 입력 필드 찾기
username_field = driver.find_element(by="name", value="username")
password_field = driver.find_element(by="name", value="password")

# username 필드에 값 입력하기
id = 'keep_selves_real'
password = 'kts71611089!'

username_field.send_keys(id)
password_field.send_keys(password)

# 로그인 버튼 클릭하기
login_button.click()

# '나중에 하기' 버튼 찾기 (XPath 사용)
later_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@role="button" and text()="나중에 하기"]')))

# '나중에 하기' 버튼 클릭하기
later_button.click()

# 알림 설정 나중에 버튼 클릭하기

In [6]:
# 날짜와 대화 메시지를 추출할 선택자 정의
date_selector = 'span.xk50ysn'
others_msg_selector = 'div.x6prxxf.x1fc57z9.x1yc453h.x126k92a.xzsf02u'
my_msg_selector = 'div.x6prxxf.x1fc57z9.x1yc453h.x126k92a.x14ctfv'

# 새로운 메시지가 온 대화창 바로 선택
new_message_windows = driver.find_elements(By.CSS_SELECTOR, '.x1i10hfl.x1qjc9v5.xjbqb8w .x6s0dn4.xzolkzo.x12go9s9.x1rnf11y')

if new_message_windows:
    for new_message_window in new_message_windows:
        new_message_window.click()
        
        # 대화 내용이 로드될 때까지 기다림
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, others_msg_selector))
        )
        
        # 현재 페이지의 HTML을 가져옴
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        recent_date = soup.select(date_selector)[-1].text
        my_messages = [msg.text for msg in soup.select(my_msg_selector)]
        others_messages = [msg.text for msg in soup.select(others_msg_selector)]

        print(others_messages[-1])
        print('-'*77)

헤이
-----------------------------------------------------------------------------


In [7]:
soup.select('div[role="row"] > div > div > div > div > h4')

[<h4 class="x1heor9g x1qlqyl8 x1pd3egz x1a2a7pz" dir="auto"><div class="xzpqnlu x1hyvwdk xqtp20y x6ikm8r x10wlt62 xnalus7">오늘 오후 3:24</div><div aria-hidden="true" class="xjpr12u xr9ek0c x2b8uid" data-scope="date_break"><span class="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1pg5gke xvq8zen xo1l8bm x12scifz" dir="auto"><span class="xk50ysn">오후 3:24</span></span></div></h4>]

In [8]:
# Initialize the dictionary to hold the conversations
convo_by_date = {}

# Extract the parent divs of dates
date_divs = soup.select('div[role="row"] > div > div > div > div > h4')

for date_div in date_divs:
    # Extract the date
    date_str = date_div.get_text()
    
    # Get the parent div of the date
    parent_div = date_div.find_parent().find_parent().find_parent()
    
    # Extract the messages within the parent div
    messages = [msg_div.get_text() for msg_div in parent_div.select('div.x6prxxf.x1fc57z9.x1yc453h.x126k92a.xzsf02u')]
    
    # Store the messages in the dictionary
    convo_by_date[date_str] = messages